In [1]:
import numpy as np
import torch
from torch import nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torchmetrics.classification import Accuracy
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.auto import tqdm

import matplotlib.pyplot as plt

import os

In [2]:
# Setup device-agnocstic code
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = '1' #LocalResponseNorm is not available on Apple MPS / this line still does not fix it so i used google colab

In [3]:
# Define dataloaders for the CIFAR 10 dataset
def get_train_valid_loader(data_dir,
                           batch_size,
                           augment,
                           random_seed,
                           valid_size=0.1,
                           shuffle=True):
    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )

    # define transforms
    valid_transform = transforms.Compose([
            transforms.Resize((227,227)),
            transforms.ToTensor(),
            normalize,
    ])
    #perform augmentation to strengthen model training
    if augment:
        train_transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(), #horizontal flip
            transforms.ToTensor(),
            normalize,
        ])
    else:
        train_transform = transforms.Compose([
            transforms.Resize((227,227)),
            transforms.ToTensor(),
            normalize,
        ])

    train_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=train_transform,
    )

    valid_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=valid_transform,
    )

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler)

    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler)

    return (train_loader, valid_loader)

def get_test_loader(data_dir,
                    batch_size,
                    shuffle=True):
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )

    # define transform
    transform = transforms.Compose([
        transforms.Resize((227,227)),
        transforms.ToTensor(),
        normalize,
    ])

    dataset = datasets.CIFAR10(
        root=data_dir, train=False,
        download=True, transform=transform,
    )

    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle
    )

    return data_loader

In [4]:
# Load CIFAR10 data
train_data, validation_data = get_train_valid_loader(data_dir='datasets', augment=False, batch_size=128, random_seed=42)
test_data = get_test_loader(data_dir='datasets', batch_size=128, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [5]:
#Create AlexNet architecture

class AlexNet(nn.Module):
    def __init__(self, in_channels, classes=10):
        super().__init__()
        self.layer_1 = nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=96,
                kernel_size=11,
                stride=4,
                padding=0
                ),
            nn.ReLU(),
            nn.LocalResponseNorm(5, alpha=10**-4, beta=0.75, k=2), # 'brightness normalization' to reduce overfitting
            nn.MaxPool2d(kernel_size=3, stride=2) #overlapping pooling
        )
        self.layer_2 = nn.Sequential(
            nn.Conv2d(
                in_channels=96,
                out_channels=256,
                kernel_size=5,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.LocalResponseNorm(5, alpha=10**-4, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.layer_3 = nn.Sequential(
            nn.Conv2d(
                in_channels=256,
                out_channels=384,
                kernel_size=3,
                stride=1,
                padding=1
                ),
            nn.ReLU()
        )
        self.layer_4 = nn.Sequential(
            nn.Conv2d(in_channels=384,
                      out_channels=384,
                      kernel_size=3,
                      stride=1,
                      padding=1
                      ),
            nn.ReLU()
        )
        self.layer_5 = nn.Sequential(
            nn.Conv2d(
                in_channels=384,
                out_channels=256,
                kernel_size=3,
                stride=1,
                padding=1
                ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.layer_6 = nn.Sequential(
            nn.Flatten(),
            #nn.Dropout(0.5),
            nn.Linear(
                in_features=256*6*6,
                out_features=4096,
                ),
            nn.ReLU()
        )
        self.layer_7 = nn.Sequential(
            #nn.Dropout(0.5),
            nn.Linear(
                in_features=4096,
                out_features=4096,
                ),
            nn.ReLU()
        )
        self.layer_8 = nn.Sequential(
            nn.Linear(
                in_features=4096,
                out_features=classes,
                ),
        )

        self._initialize_weights()

    def forward(self, x):
        x = self.layer_1(x)
        x = self.layer_2(x)
        x = self.layer_3(x)
        x = self.layer_4(x)
        x = self.layer_5(x)
        x = self.layer_6(x)
        x = self.layer_7(x)
        x = self.layer_8(x)
        return x


    def _initialize_weights(self):
        biases = [1, 3, 4, 5, 6, 7]
        for i in self.modules():
            if isinstance(i, (nn.Conv2d, nn.Linear)):
                nn.init.normal_(i.weight, mean=0, std=0.01),
                if i in biases:
                    nn.init.constant_(i.bias, 1)
                else:
                    nn.init.constant_(i.bias ,0)




In [6]:
#initialize neural network
net = AlexNet(in_channels=3, classes=10).to(device=device)

In [7]:
#define loss function, optimizer and accuracy function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    params=net.parameters(),
    momentum=0.9,
    weight_decay=0.0005,
    lr=0.01
    )
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10) #divide lr by 10 everytime validation loss plateaus
accuracy_fn = Accuracy(task='multiclass', num_classes=10).to(device)


In [8]:
# training step
epochs = 90

torch.manual_seed(42)
for epoch in tqdm(range(epochs), desc="Epochs"):
    train_loss = 0
    print(f"Epoch: {epoch}/{epochs}\n-----------------------")
    for i, (images, labels) in tqdm(enumerate(train_data), total=len(train_data), desc="Batches"):
        train_loss = 0
        images = images.to(device)
        labels = labels.to(device)

        #training
        net.train()

        #forward pass
        train_pred = net(images)

        #calculate loss
        train_loss += loss_fn(train_pred, labels)

        #backprop
        optimizer.zero_grad()
        train_loss.backward()

        #gradient descent
        optimizer.step()

    train_loss /= len(train_data)

    #evaluation
    eval_loss, eval_acc = 0, 0
    net.eval()
    with torch.inference_mode():
        for i, (images, labels) in enumerate(validation_data):
            images = images.to(device)
            labels = labels.to(device)
            #forward pass
            eval_logits = net(images)
            eval_pred = eval_logits.argmax(dim=1)

            #calculate loss and accuracy
            eval_loss += loss_fn(eval_logits, labels)
            eval_acc += accuracy_fn(eval_pred, labels)

        eval_loss /= len(validation_data)
        eval_acc /= len(validation_data)
        eval_acc *= 100

    print(f"\nTrain loss: {train_loss:.10f} | Val loss: {eval_loss:.10f}, Val acc: {eval_acc:.2f}%\n")
    scheduler.step(eval_loss)


Epochs:   0%|          | 0/90 [00:00<?, ?it/s]

Epoch: 0/90
-----------------------


Batches:   0%|          | 0/352 [00:00<?, ?it/s]


Train loss: 0.0047697062 | Val loss: 1.8508930206, Val acc: 33.48%

Epoch: 1/90
-----------------------


Batches:   0%|          | 0/352 [00:00<?, ?it/s]

...


Train loss: 0.0000019617 | Val loss: 1.1159338951, Val acc: 80.94%

Epoch: 88/90
-----------------------


Batches:   0%|          | 0/352 [00:00<?, ?it/s]


Train loss: 0.0000010176 | Val loss: 1.1588777304, Val acc: 80.64%

Epoch: 89/90
-----------------------


Batches:   0%|          | 0/352 [00:00<?, ?it/s]


Train loss: 0.0000010846 | Val loss: 1.1338763237, Val acc: 80.35%



In [10]:
net.eval()
test_acc = 0
with torch.inference_mode():
    for images, labels in test_data:
        images = images.to(device)
        labels = labels.to(device)
        
        predictions = net(images).argmax(dim=1)
        test_acc += accuracy_fn(predictions, labels)
    test_acc = (test_acc / len(test_data)) *100
    
    print(f"Accuracy of AlexNet on test images: {test_acc:.2f}%")

Accuracy of AlexNet on test images: 80.36%
